# REST Inference

### Setup

In [ ]:
!pip install opencv-python-headless

In [ ]:
infer_url = 'http://modelmesh-serving:8008/v2/models/resnet/infer'

### Image Preprocessing Functions

First, we need to preprocess and format the image.

In [ ]:
import numpy as np
import cv2

def normalize(img_data):
    """Normalize to adjust to model training dataset
    """
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        # for each pixel in each channel, divide the value by 255 to get value between [0, 1] and then normalize
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
    return norm_img_data



def preprocess(img_data, size=224):
    img = np.frombuffer(img_data, dtype=np.uint8)
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)  # BGR format

    # format of data is HWC
    # add image preprocessing if needed by the model
    img = cv2.resize(img, (224, 224))
    img = img.astype('float32')
    #convert to NCHW
    img = img.transpose(2,0,1)

    norm_img_data = normalize(img)
    reshaped_img_data = norm_img_data.reshape(1,3,size,size)
    # print(path, reshaped_img_data.shape, "; data range:",np.amin(reshaped_img_data),":",np.amax(reshaped_img_data))
    return reshaped_img_data

def getJpeg(path):
    with open(path, mode='rb') as file:
        file_content = file.read()

    return file_content

### Request Function

Builds and submits our gRPC request.

In [ ]:
# import json
import time
import requests
import classes

def rhods_rest_request(img_data):
    im_json = img_data.tolist() # Converts the array to a nested list

    data = {
        "inputs": [
            {
             "name": "gpu_0/data_0",
             "shape": [1,3,224,224],
             "datatype": "FP32",
             "data": im_json
            }
        ]
    }


    t1 = time.time()
    response = requests.post(infer_url, json=data)
    t2 = time.time()
    inference_time = t2-t1

    raw_output = response.json()    
    response_dict = response.json()
    result_arr = np.array(response_dict['outputs'][0]['data'])
    max = np.argmax(result_arr)

    return classes.imagenet_classes[max], result_arr[max]

In [ ]:
from IPython.display import Image
import classes

image_path = 'images/bee.jpeg'
Image(filename=image_path)

img_bytes = getJpeg(image_path)
img_data = preprocess(img_bytes)
image_classification, score = rhods_rest_request(img_data)

print(f"class: {image_classification}")
print(f"score: {score}")
Image(filename=image_path)